# Oppgave 4 Dataanalyse

In [105]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import plotly.express as px
from prettytable import PrettyTable

In [115]:

client_id = "aff7c34e-993d-4132-81bc-1df3e81d7868" 
end_date = datetime.today().date()
start_date = end_date - timedelta(days=365)  
station_id = "SN18700"


Funksjonen fetch_and_analyze_weather henter værdata for en spesifisert stasjon og periode som kan velges i kodesnutten over. Den analyserer fire forskjellige værvariabler: temperatur, fuktighet, nedbør og vindhastighet. For hver variabel hentes gjennomsnitt, median og standardavvik for dataene, og eventuelle feil håndteres dersom det ikke er tilgjengelig data.
- Hentede data: For hver værvariabel (som definert i elements) gjør funksjonen et API-kall til MET.no sin Frost API for å hente data.
- Statistiske beregninger: For hver type værdata beregnes gjennomsnitt, median og standardavvik. Hvis data mangler, vises det som "Ingen data".
- Tabellformat: Resultatene vises i en tabell ved hjelp av PrettyTable, som gjør det lettere å visualisere og analysere de statistiske resultatene for hver variabel.

In [ ]:
def fetch_and_analyze_weather(client_id, station_id, start_date, end_date):

    elements = [
        ("mean(air_temperature P1D)", "Temperatur", "°C"),
        ("mean(relative_humidity P1D)", "Fuktighet", "%"),
        ("sum(precipitation_amount P1D)", "Nedbør", "mm"),
        ("mean(wind_speed P1D)", "Vindhastighet", "m/s"),
    ]

    results = []

    for element_id, name, unit in elements:
        # Henter dataen fra API
        url = "https://frost.met.no/observations/v0.jsonld"
        params = {
            "sources": station_id,
            "elements": element_id,
            "referencetime": f"{start_date}/{end_date}"
        }

        try:
            response = requests.get(url, params=params, auth=(client_id, ""), timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Feil ved henting av {name}: {e}")
            results.append((name, unit, "Feil", "Feil", "Feil"))
            continue

        data = response.json()
        values = []

        # Trekker ut verdier
        for obs in data.get("data", []):
            for el in obs.get("observations", []):
                if el["elementId"] == element_id and "value" in el:
                    values.append(el["value"])

        if values:
            mean_val = round(np.mean(values), 2)
            median_val = round(np.median(values), 2)
            std_val = round(np.std(values), 2)
        else:
            mean_val = median_val = std_val = "Ingen data"

        results.append((name, unit, mean_val, median_val, std_val))

    # Lager en fin tabell
    table = PrettyTable()
    table.field_names = ["Element", "Enhet", "Gjennomsnitt", "Median", "Std.avvik"]
    for row in results:
        table.add_row(row)

    print(table)

fetch_and_analyze_weather(client_id, station_id, start_date, end_date)

+---------------+-------+--------------+--------+-----------+
|    Element    | Enhet | Gjennomsnitt | Median | Std.avvik |
+---------------+-------+--------------+--------+-----------+
|   Temperatur  |   °C  |     8.29     |  8.3   |    7.48   |
|   Fuktighet   |   %   |    77.83     |  79.0  |   15.63   |
|     Nedbør    |   mm  |     2.49     |  0.0   |    5.56   |
| Vindhastighet |  m/s  |     2.42     |  2.3   |    1.0    |
+---------------+-------+--------------+--------+-----------+


# Oppgave 5 Visualisering

Funksjonen fetch_weather_data henter værdata fra MET.no sin Frost API for en spesifisert stasjon og tidsperiode. Den tar en liste av elementer (værvariabler) som parameter, og lager et API-kall med disse.
- API-kall: Bruker requests for å sende et GET-kall til API-en med de spesifiserte parameterne.
- Feilhåndtering: Hvis det oppstår en HTTP-feil eller en annen type forespørselsfeil, håndteres disse ved å skrive ut en feilmelding, og funksjonen returnerer None ved feil.
- Returnering av data: Hvis API-kallet lykkes, returneres JSON-dataene som inneholder værinformasjonen.

In [ ]:
def fetch_weather_data(client_id, station_id, start_date, end_date, elements):
    url = "https://frost.met.no/observations/v0.jsonld"
    params = {
        "sources": station_id,
        "elements": ",".join(elements),
        "referencetime": f"{start_date}/{end_date}"
    }

    try:
        response = requests.get(url, params=params, auth=(client_id, ""), timeout=10)
        response.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error: {http_err}")
        print("Response text:", response.text)
        return None
    except requests.exceptions.RequestException as err:
        print(f"Error fetching data: {err}")
        return None

    return response.json()
    

Funksjonen parse_weather_data tar inn API-responsen (data) og et element_id som parameter. Den søker gjennom værobservasjonene for det spesifikke elementet (som temperatur, vindhastighet, etc.) og trekker ut relevante data.
- Feilhåndtering: Hvis dataene ikke er tilgjengelige eller API-responsen er tom, returneres en tom DataFrame med en feilmelding.
- Dataforberedelse: Funksjonen konverterer datoene til datetime-format, sorterer dataene etter dato og returnerer en DataFrame med dato og verdi for det ønskede elementet.

In [109]:
def parse_weather_data(data, element_id):
    if not data or "data" not in data:
        print("No data returned from API.")
        return pd.DataFrame()

    observations = []
    for obs in data["data"]:
        for element in obs["observations"]:
            if element["elementId"] == element_id:
                observations.append({
                    "date": obs["referenceTime"],
                    "value": element["value"]
                })

    df = pd.DataFrame(observations)
    if df.empty:
        print("No data available for this element.")
        return df

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")
    return df


Funksjonen plot_weather_data tar inn en DataFrame med værdata, navnet på elementet, stasjonsnavnet, start- og sluttdato, samt en enhet. Den bruker Plotly for å lage et linjediagram med tid (dato) på x-aksen og den målte verdien på y-aksen.
- Plot Layout: Tittelen på grafen inkluderer elementnavn, stasjon og datoperiode, og aksetitlene tilpasses automatisk.
- Hover-Info: Når brukeren holder musen over et punkt på grafen, vises dato og verdi med enheten.
- Feilhåndtering: Hvis DataFrame er tom, avsluttes funksjonen uten å lage plottet.

In [110]:
def plot_weather_data(df, element_name, station_name, start_date, end_date, unit=""):
    if df.empty:
        return

    fig = px.line(
        df,
        x="date",
        y="value",
        title=f"{element_name} – {station_name} ({start_date} til {end_date})",
        labels={"date": "Dato", "value": f"{element_name} ({unit})"},
        template="plotly_white"
    )

    fig.update_traces(
        mode="lines",
        hovertemplate="Dato: %{x|%Y-%m-%d}<br>Verdi: %{y:.1f} {unit}"
    )
    fig.update_layout(
        xaxis_title="Dato",
        yaxis_title=f"{element_name} ({unit})",
        hovermode="x unified",
        showlegend=False
    )

    fig.show()


Videre tar vi i bruk fetch_weather_data, parse_weather_data og plot_weather_data for å hente, prosessere og visualisere værdataene for tre forskjellige meteorologiske elementer: gjennomsnittlig temperatur, relativ fuktighet og vindhastighet. Til slutt, for å visualisere nedbør, benytter vi et barplot i stedet for en linjegraf. Dette gir en bedre fremstilling av nedbøren, ettersom nedbør typisk er akkumulert over tid og er mer egnet for å vises i form av separate søyler per dag. For nedbøren bruker vi funksjonen plot_weather_Barplot, som er tilpasset for denne typen visualisering.

In [111]:

element_id = "mean(air_temperature P1D)"
element_name = "Gjennomsnittlig temperatur"
unit = "C"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)


In [112]:
element_id = "mean(relative_humidity P1D)"
element_name = "Relativ fuktighet"
unit = "%"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)

In [113]:
element_id = "mean(wind_speed P1D)"
element_name = "Vindhastighet"
unit = "m/s"

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_data(df, element_name, "Oslo - Blindern", start_date, end_date, unit)

In [114]:
element_id = "sum(precipitation_amount P1D)"
element_name = "Nedbør"
unit = "mm"

def plot_weather_Barplot(df, element_name, station_name, start_date, end_date, unit=""):
    if df.empty:
        print("Ingen data å vise.")
        return

    fig = px.bar(
        df,
        x="date",
        y="value",
        title=f"{element_name} - {station_name} ({start_date} til {end_date})",
        labels={"date": "Dato", "value": f"{element_name} ({unit})"},
        template="plotly_white"
    )

    fig.update_traces(
        hovertemplate="Dato: %{x|%Y-%m-%d}<br>Verdi: %{y:.1f} {unit}"
    )
    fig.update_layout(
        xaxis_title="Dato",
        yaxis_title=f"{element_name} ({unit})",
        hovermode="x unified",
        showlegend=False
    )

    fig.show()

json_data = fetch_weather_data(client_id, station_id, start_date, end_date, [element_id])
df = parse_weather_data(json_data, element_id)
plot_weather_Barplot(df, element_name, "Oslo - Blindern", start_date, end_date, unit)

# Oppgave 6 Predictiv analyse